In [3]:
!pip install nltk 
import os, re, pickle, joblib
import numpy as np
import pandas as pd
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import openpyxl
import html

# --- nltk resources (local) ---

needed = [
    ("tokenizers/punkt", "punkt"),
    ("tokenizers/punkt_tab", "punkt_tab"),   # <-- thêm cái này
    ("corpora/stopwords", "stopwords"),
]

for path, pkg in needed:
    try:
        nltk.data.find(path)
    except LookupError:
        nltk.download(pkg)

STOPWORDS = set(stopwords.words("english"))



   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 6.7 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [5]:

# --- file input ---
TEST_FILE = r"C:\Users\ADMIN\CS221---HateSpeech-Detection-Project\Inference_3_pipeline\test_100_stratified_from_test.csv"   
SHEET_NAME = None                # ví dụ: "Sheet1" nếu cần

if TEST_FILE.lower().endswith(".xlsx"):
    tmp = pd.read_excel(TEST_FILE, sheet_name=SHEET_NAME)  # SHEET_NAME có thể None
    # Nếu sheet_name=None => tmp là dict {sheet: df}
    if isinstance(tmp, dict):
        # lấy sheet đầu tiên
        first_sheet = list(tmp.keys())[0]
        print("Detected multiple sheets. Using sheet:", first_sheet)
        df = tmp[first_sheet]
    else:
        df = tmp
else:
    df = pd.read_csv(TEST_FILE)

print("df type:", type(df))
print("columns:", list(df.columns))

# --- columns ---
TEXT_COL = "Tweet"
LABEL_COL = "Final Votes"  # nếu không có label: set = None

assert TEXT_COL in df.columns, f"Thiếu cột '{TEXT_COL}' trong file test!"

# --- nltk resources (local) ---
for pkg in ["punkt", "stopwords"]:
    try:
        nltk.data.find(pkg)
    except LookupError:
        nltk.download(pkg)

STOPWORDS = set(stopwords.words("english"))

def clean_text(text):
    text = str(text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)                   # Remove mentions
    text = re.sub(r'#\w+', '', text)                   # Remove hashtags
    text = text.lower()                                # Lowercase
    tokens = word_tokenize(text)                       # Tokenize
    filtered_tokens = [
        w for w in tokens
        if w.isalpha() and w not in STOPWORDS          # Remove stopwords + non-alpha
    ]
    return " ".join(filtered_tokens)

URL_RE = re.compile(r'https?://\S+|www\.\S+')
USER_RE = re.compile(r'@\w+')
HASHTAG_RE = re.compile(r'#(\w+)')  # giữ chữ trong hashtag

def clean_text_bert(text):
    text = str(text)
    text = html.unescape(text)

    # Thay vì xóa sạch, dùng token để giữ tín hiệu "có URL/mention"
    text = URL_RE.sub(' [URL] ', text)
    text = USER_RE.sub(' [USER] ', text)

    # Giữ nội dung hashtag, chỉ bỏ dấu #
    text = HASHTAG_RE.sub(r' \1 ', text)

    # Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()

    # Chỉ bật nếu bạn dùng model uncased (vd: bert-base-uncased)
    text = text.lower()

    return text

# --- apply cleaning ---
df["tweet_clean"] = df[TEXT_COL].apply(clean_text)

print("Data after cleaning:")
print(df[[TEXT_COL, "tweet_clean"]].head())

# list texts to feed models
texts = df[TEXT_COL].apply(clean_text).astype(str).fillna("").tolist()
texts_bert = df[TEXT_COL].apply(clean_text_bert).astype(str).fillna("").tolist()


df type: <class 'pandas.core.frame.DataFrame'>
columns: ['Tweet', 'Final Votes']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data after cleaning:
                                               Tweet  \
0  RT @BirdGang316: If you a bird throw it up #bi...   
1  RT @TheRaceDraft: Karate bitch https://t.co/Ut...   
2  When ratchet bitches find out the club ain't 1...   
3  RT @trevso_electric: You want a hot body? You ...   
4                    We don't pop out bitch we slide   

                                         tweet_clean  
0                                      rt bird throw  
1                                    rt karate bitch  
2                   ratchet bitches find club ai get  
3  rt want hot body want bugatti better born weal...  
4                                    pop bitch slide  


In [7]:
# ---------- 1) Paths tới 3 pipeline artifacts ----------
# ML artifacts (từ notebook: tfidf.pkl + *_tuned.pkl)
ML_DIR   = "random_forest_models"  
PIPE_ML_PKL = r"C:\Users\ADMIN\CS221---HateSpeech-Detection-Project\outputs_ML\pipe_rf.pkl"

# RNN artifacts (từ notebook: tokenizer_rnn.pkl + *.keras)
RNN_DIR = "rnn_lstm_models"  
TOKENIZER_PKL   = r"C:\Users\PC\CS221\output_RNN_LSTM\LSTM1D_model\tokenizer_lstm1d.pkl"
RNN_MODEL_KERAS = r"C:\Users\PC\CS221\output_RNN_LSTM\LSTM1D_model\lstm1d_hate_speech.keras"

# BERT artifacts (từ notebook: checkpoints/bert_best)
BERT_DIR = r"C:\Users\ADMIN\CS221---HateSpeech-Detection-Project\outputs_bert\bert_best"

In [8]:
def normalize_label(x: str) -> str:
    x = str(x).strip().lower()
    if x in ["hate", "hate_speech", "hate speech", "hatespeech"]:
        return "Hate Speech"
    if x in ["offensive", "offensive_language", "offensive language", "offensivelanguage"]:
        return "Offensive"
    if x in ["neither", "neutral", "none", "clean"]:
        return "Neither"
    return str(x)

ID2LABEL_ML = {0: "Hate Speech", 1: "Offensive", 2: "Neither"}

In [9]:
def predict_ml(texts, pipe_path=PIPE_ML_PKL):
    """
    Load 1 sklearn Pipeline (.pkl) đã chứa tfidf + classifier.
    Return:
      - preds: list[str] (Hate/Offensive/Neither hoặc label gốc của model)
      - proba: np.ndarray shape (n,3) hoặc None nếu model không hỗ trợ
    """
    pipe = joblib.load(pipe_path)

    # sklearn pipeline có thể predict ra string labels luôn (vì bạn train y là "Hate"/"Offensive"/"neither")
    y_pred = pipe.predict(texts)

    # normalize về chuẩn bạn dùng trong report
    preds = [normalize_label(x) for x in y_pred]

    proba = None
    if hasattr(pipe, "predict_proba"):
        proba = pipe.predict_proba(texts)
    return preds, proba

In [10]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

def predict_bert(texts, batch_size=32, max_len=256):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    tokenizer = BertTokenizer.from_pretrained(BERT_DIR)
    model = BertForSequenceClassification.from_pretrained(BERT_DIR)
    model.to(device)
    model.eval()

    all_preds, all_probs = [], []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(
            batch,
            truncation=True,
            padding=True,
            max_length=max_len,
            return_tensors="pt"
        )
        enc = {k: v.to(device) for k, v in enc.items()}
        with torch.no_grad():
            logits = model(**enc).logits
            probs = torch.softmax(logits, dim=-1).cpu().numpy()
            pred_ids = probs.argmax(axis=1)

        # theo notebook BERT: label2id = {'hate_speech':0,'offensive_language':1,'neither':2}
        ID2LABEL_BERT = {0: "Hate Speech", 1: "Offensive", 2: "Neither"}
        all_preds.extend([ID2LABEL_BERT[int(j)] for j in pred_ids])
        all_probs.append(probs)

    return all_preds, np.vstack(all_probs)

c:\Users\ADMIN\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ADMIN\miniconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [15]:
print("len(df) =", len(df))
print("len(pred_ml)  =", len(pred_ml))
print("len(pred_rnn) =", len(pred_rnn))
print("len(pred_bert)=", len(pred_bert))


len(df) = 100
len(pred_ml)  = 100
len(pred_rnn) = 100
len(pred_bert)= 100


In [14]:
# %% 
from sklearn.metrics import classification_report, accuracy_score, f1_score
import numpy as np
import pandas as pd

RNN_PREDS_CSV = r"C:\Users\ADMIN\CS221---HateSpeech-Detection-Project\rnn_preds.csv"   # file bạn đã export

# 1) ML + BERT inference
pred_ml, proba_ml     = predict_ml(texts)
pred_bert, proba_bert = predict_bert(texts_bert)

# 2) Load RNN preds từ CSV rồi merge theo thứ tự dòng
rnn_df = pd.read_csv(RNN_PREDS_CSV)

# cố gắng đoán tên cột dự đoán trong file rnn_preds.csv
cand_cols = [c for c in ["pred_RNN", "rnn_pred", "pred", "prediction", "label"] if c in rnn_df.columns]
if len(cand_cols) == 0:
    # fallback: lấy cột đầu tiên
    rnn_col = rnn_df.columns[0]
else:
    rnn_col = cand_cols[0]

pred_rnn = rnn_df[rnn_col].astype(str).tolist()

# check số dòng khớp
assert len(pred_rnn) == len(df), f"RNN preds rows ({len(pred_rnn)}) != df rows ({len(df)})"

# 3) Build output table
out = df.copy()
out["pred_ML"]   = pred_ml
out["pred_RNN"]  = pred_rnn
out["pred_BERT"] = pred_bert

# normalize label thật nếu có
if LABEL_COL and (LABEL_COL in out.columns):
    out["y_true"] = out[LABEL_COL].apply(normalize_label)

# 4) Save merged predictions
out_path = r"C:\Users\ADMIN\CS221---HateSpeech-Detection-Project\preds_3pipelines_dataset.csv"
out.to_csv(out_path, index=False)
print("Saved:", out_path)

# 5) Evaluation (nếu có y_true)
if "y_true" in out.columns:
    y_true = out["y_true"].tolist()
    labels = ["Hate Speech", "Offensive", "Neither"]

    def eval_one(name, y_pred):
        print(f"\n=== {name} ===")
        print("Accuracy:", round(accuracy_score(y_true, y_pred), 4))
        print("Macro-F1:", round(f1_score(y_true, y_pred, average="macro"), 4))
        print(classification_report(y_true, y_pred, labels=labels, digits=4))

    eval_one("ML",   out["pred_ML"].tolist())
    eval_one("RNN",  out["pred_RNN"].tolist())
    eval_one("BERT", out["pred_BERT"].tolist())

# 6) Disagreement analysis (không cần y_true)
out["all_agree"] = (out["pred_ML"] == out["pred_RNN"]) & (out["pred_RNN"] == out["pred_BERT"])
disagree = out[~out["all_agree"]].copy()

disagree_path = r"C:\Users\ADMIN\CS221---HateSpeech-Detection-Project\disagree_cases_dataset.csv"
disagree.to_csv(disagree_path, index=False)
print("Saved:", disagree_path, "| n_disagree =", len(disagree))


c:\Users\ADMIN\miniconda3\Lib\site-packages\sklearn\base.py:463: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.4.2 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ADMIN\miniconda3\Lib\site-packages\sklearn\base.py:463: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.4.2 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ADMIN\miniconda3\Lib\site-packages\sklearn\base.py:463: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.2 when using version 1.8.0. This might le

Saved: C:\Users\ADMIN\CS221---HateSpeech-Detection-Project\preds_3pipelines_dataset.csv

=== ML ===
Accuracy: 0.74
Macro-F1: 0.6054
              precision    recall  f1-score   support

 Hate Speech     0.5000    0.3333    0.4000         6
   Offensive     0.9492    0.7273    0.8235        77
     Neither     0.4324    0.9412    0.5926        17

    accuracy                         0.7400       100
   macro avg     0.6272    0.6673    0.6054       100
weighted avg     0.8344    0.7400    0.7589       100


=== RNN ===
Accuracy: 0.89
Macro-F1: 0.7784
              precision    recall  f1-score   support

 Hate Speech     0.3636    0.6667    0.4706         6
   Offensive     0.9459    0.9091    0.9272        77
     Neither     1.0000    0.8824    0.9375        17

    accuracy                         0.8900       100
   macro avg     0.7699    0.8194    0.7784       100
weighted avg     0.9202    0.8900    0.9015       100


=== BERT ===
Accuracy: 0.95
Macro-F1: 0.8464
              p